In [ ]:
import numpy as np 
from scipy.signal import butter, lfilter, freqz, filtfilt
import matplotlib.pyplot as plt 
%matplotlib inline
import sys
sys.path.append('../')
import utils as ut

In [ ]:
folder = '/Users/Jan/Dropbox/Master/LR_Kuehn/data/dystonia_rest/for_python/good_theta/'
file = 'spmeeg_13.mat'
d = ut.load_data_spm(file, folder)

In [ ]:
lfp = d['data'][1]
fs = d['fsample'][0][0]

In [ ]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def butter_bandpass_filter2(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

In [ ]:
low = 2. 
high = 14. 
plt.figure(figsize=(10, 5))
for order in [2, 3, 4, 5, 6]:
    b, a = butter_bandpass(low, high, fs, order=order)
    w, h = freqz(b, a, worN=2000)
    up_to = 400
    plt.plot(((fs * 0.5 / np.pi) * w)[:up_to], abs(h)[:up_to], label="order = %d" % order)
plt.legend()
plt.title('Frequency response')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Gain')
plt.grid(True)
plt.legend(loc='best');

### Look at the shape of the frequency response before filtering!

dependent on the frequency range, the order of the filter that produces a good response is different. 

For the theta range it is order 4,

for the beta range order 5 

higher order give very different frequency response 

In [ ]:
low = 4. 
high = 12. 
plt.figure(figsize=(15, 10))
start = 1000
stop = start + 700
lfp = lfp - lfp.mean()
plt.plot(lfp[start:stop])
for order in [2, 3, 4, 5]:
    y = butter_bandpass_filter(lfp, low, high, fs, order=order)
    plt.plot(y[start:stop], label="order = %d" % order)
plt.legend();

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(lfp[start:stop])
for order in [2, 3, 4, 5]:
    y = butter_bandpass_filter2(lfp, low, high, fs, order=order)
    plt.plot(y[start:stop], label="order = %d" % order)
plt.legend();

### The effect of using filtfilt 

filtfilt applies the filter forward and backward to improve the response. we can see that the lower orders result in very similar filtered signal, whereas with using lfilter they are quite different. Thus, filtfilt seems to improce the filtering for these orders. 

However, for order 5, the filtfilt result is suddenly very different, what is going on? 